In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from gaussian_renderer import network_gui
from sgs2.gaussian import GaussianModel
from sgs2.scene import Scene
from sgs2.trainers.grid_trainer import GridTrainer
from sgs2.trainers.simple_trainer import SimpleTrainer
from sgs2.evaluation import evaluate_scene
import os
network_gui.init("127.0.0.1", 6009)

Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]


/home/heng/miniconda3/envs/gsn/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/heng/miniconda3/envs/gsn/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /home/heng/miniconda3/envs/gsn/lib/python3.8/site-packages/lpips/weights/v0.1/vgg.pth
('127.0.0.1', 6009)


In [3]:
def make_iteration_callback(scene, folder_path="./output", save_interval=100, name="gaussian_model"):
    os.makedirs(folder_path, exist_ok=True)
    def iteration_callback(iteration, n_gauss, mem_use, model=None):
        if model and (iteration % save_interval == 0):
            model_path = os.path.join(folder_path, f"{name}_{iteration}")
            print(f"Saving model at iteration {iteration} to {model_path}")
            model.export_for_sibr(scene, model_path)
    return iteration_callback

# Dataset 1: Train scene
- Method 1: Simple trainer (10K iterations)
- Method 2: Grid trainer, no loss mask (10K iterations)
- Method 3: Grid trainer, with loss mask (10K iterations)

In [ ]:
# Load train scene
train_scene = Scene(source_path="./datasets/train/")

In [ ]:
# Method 1
model = GaussianModel.create_from_scene(train_scene)
simple_trainer = SimpleTrainer(iterations=10000, iteration_callback=make_iteration_callback(train_scene, name="train_scene_simple", save_interval=1000))
simple = simple_trainer.train(train_scene, model)

In [ ]:
grid_model = GaussianModel.create_from_scene(train_scene)
grid_trainer = GridTrainer(iterations=10000, network_gui=True, clean_chunk_edges=True, chunk_loss_masking=False)
grid = grid_trainer.train(train_scene, grid_model)


In [ ]:
grid.export_for_sibr(train_scene, "./output/train_scene_grid_10000")

In [4]:
# Now, we train for "./datasets/alleyds/"
alley_scene = Scene(source_path="./datasets/alleyds/")
alley_model = GaussianModel.create_from_scene(alley_scene)

Reading camera 346/346


/home/heng/miniconda3/envs/gsn/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:866: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


Loading 346 Train Cameras at scale 1.0
Loading Test Cameras
Number of points at initialisation :  507378


In [ ]:
# Train using simple trainer
simple_trainer = SimpleTrainer(iterations=10000, iteration_callback=make_iteration_callback(alley_scene, name="alley_scene_simple", save_interval=1000))
simple = simple_trainer.train(alley_scene, alley_model)

In [5]:
# Train with grid, without chunk loss masking
grid_model = GaussianModel.create_from_scene(alley_scene)
grid_trainer = GridTrainer(iterations=10000, network_gui=True, clean_chunk_edges=True, chunk_loss_masking=True)
grid = grid_trainer.train(alley_scene, grid_model)
grid.export_for_sibr(alley_scene, "./output/alley_scene_grid_loss_masking_10000")

Number of points at initialisation :  507378
Pre-training gaussians...


100%|██████████| 1000/1000 [00:33<00:00, 30.25it/s]


Computing visibility...


100%|██████████| 8/8 [00:04<00:00,  1.64it/s]


Training submodel 1/7...
Filtered cameras from 346 to 346


...

Training submodel 3/7...
Filtered cameras from 346 to 2


100%|██████████| 10000/10000 [02:19<00:00, 71.45it/s]


Training submodel 4/7...
Filtered cameras from 346 to 18


100%|██████████| 10000/10000 [02:22<00:00, 70.10it/s]


Training submodel 5/7...
No cameras meet the visibility threshold of 0.05
Training submodel 6/7...
No cameras meet the visibility threshold of 0.05
Training submodel 7/7...
No cameras meet the visibility threshold of 0.05
Combining gaussians...
Done.
Creating output folder at /home/heng/Documents/GitHub/gaussian-splatting/output/alley_scene_grid_loss_masking_10000


In [ ]:
# Train with grid, without chunk loss masking
grid_model = GaussianModel.create_from_scene(alley_scene)
grid_trainer = GridTrainer(iterations=10000, network_gui=True, clean_chunk_edges=True, chunk_loss_masking=False)
grid = grid_trainer.train(alley_scene, grid_model)
grid.export_for_sibr(alley_scene, "./output/alley_scene_grid_10000")

In [ ]:
# Train with grid, with chunk loss masking
grid_model = GaussianModel.create_from_scene(alley_scene)
grid_trainer = GridTrainer(iterations=10000, network_gui=True, clean_chunk_edges=True, chunk_loss_masking=True)
grid = grid_trainer.train(alley_scene, grid_model)
grid.export_for_sibr(alley_scene, "./output/alley_scene_grid_10000_chunk_masking")